In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install speechrecognition

     |████████████████████████████████| 32.8 MB 1.4 MB/s 


In [ ]:
import librosa
import numpy as np
def extract(file_name):
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

In [ ]:
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
#import feature
import speech_recognition as sr

def predict(filename):
    fmodel_path = '/content/drive/MyDrive/Main Project/Emotion_Voice_Detection_Model.h5'
    result=[]
    xtr = joblib.load('/content/drive/MyDrive/Main Project/X.train')
    scaler = StandardScaler()
    scaler.fit_transform(xtr)
    clf2 = joblib.load(fmodel_path)
    j=np.array(extract(filename))
    k = np.expand_dims(j, axis=0)
    k = scaler.transform(k)
    result.append(clf2.predict(k)[0])

    r = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = r.record(source)
        try:
            text = r.recognize_google(audio_data)
        except sr.UnknownValueError:  
            text="*I could not understand audio*" 
        result.append(text)
    print(result)

In [ ]:
predict("/content/drive/MyDrive/Main Project/speech/Actor_14/03-01-01-01-01-01-14.wav")

['neutral_female', 'kids are talking by the door']


In [ ]:
predict("/content/drive/MyDrive/Main Project/test.wav")

['surprised_female', 'dogs are sitting by the door']


In [ ]:
predict("/content/drive/MyDrive/Main Project/Monday at 3-07 pm.wav")

['fearful_male', 'I am very afraid']
